In [1]:
import pandas as pd
import requests
import xml.etree.ElementTree as ET
from bs4 import BeautifulSoup
from datetime import datetime, timedelta
from tqdm import tqdm

In [2]:
url = "https://www.airportal.go.kr/knowledge/statsnew/realtime/abc.SheetAction"

# create an empty list to store the data
data = []

# loop through each day in 2022 and retrieve the data
start_date = datetime(2022, 1, 1)
end_date = datetime(2023, 4, 1)
delta = timedelta(days=1)

for date in tqdm(pd.date_range(start_date, end_date, freq='D')):
    # set the current date in the parameters
    current_date = date.strftime('%Y%m%d')
    parameters = {
        "S_CONTROLLER": "aipsPack.statnuri.Stat_RealTime_Data2",
        "S_METHOD": "searchAirlineRoute",
        "S_SAVENAME": "AP_ROUTE|AP1|AP2|AL_ICAO|DEP_FP|ARR_FP|TOTAL_FP|DEP_PERSON|ARR_PERSON|_PERSON|DEP_WEIGHT|ARR_WEIGHT|TOTAL_WEIGHT",
        "S_FORWARD": "",
        "S_HEADSTR": "",
        "current_dt_from": current_date,
        "current_dt_to": current_date,
        "seah_ad": "RKPC",  # 제주공항
        "srch_al": "KR",
        "ibTabTop0": "",
        "editpage0": "",
        "ibTabBottom0": ""
    }

    response = requests.get(url, parameters)

    # assuming the response is stored in a variable called 'response'
    soup = BeautifulSoup(response.content, 'html.parser')

    # find all the 'TR' tags in the response
    rows = soup.find_all('tr')

    # loop through each row and extract the data
    for row in rows:
        # find all the 'TD' tags in the row
        cells = row.find_all('td')
        # extract the text from each cell and append to the data list
        data.append([current_date] + [cell.text for cell in cells])

# create a pandas dataframe from the data
df = pd.DataFrame(data, columns=["DT", "DEPART", "ARRIVE", "DEPART2", "AIRLINE", "AIR_DEPART", "AIR_ARRIVE", "AIR_SUM", "PSG_DEPART", "PSG_ARRIVE", "PSG_SUM", "CARGO_DEPART", "CARGO_ARRIVE", "CARGO_SUM"])

df["ARRIVE"] = df["ARRIVE"].str.extract(r'\((.*?)\)')

df = df.drop(["DEPART2", "AIR_SUM", "PSG_SUM", "CARGO_DEPART", "CARGO_ARRIVE", "CARGO_SUM"], axis=1)

df = df.astype({
    "DT": "datetime64"
    , "DEPART" : "string"
    , "ARRIVE" : "string"
    , "AIRLINE" : "string"
    , "AIR_DEPART" : "int64"
    , "AIR_ARRIVE" : "int64"
    , "PSG_DEPART" : "int64"
    , "PSG_ARRIVE" : "int64"
})

# print the dataframe
df


100%|██████████| 456/456 [01:35<00:00,  4.80it/s]


,DT,DEPART,ARRIVE,AIRLINE,AIR_DEPART,AIR_ARRIVE,PSG_DEPART,PSG_ARRIVE
0,2022-01-01,RKJJ,RKPC,AAR,3,3,427,330
1,2022-01-01,RKJJ,RKPC,JJA,4,4,673,516
2,2022-01-01,RKJJ,RKPC,JNA,3,3,425,349
3,2022-01-01,RKJJ,RKPC,KAL,2,2,202,180
4,2022-01-01,RKJJ,RKPC,TWB,3,3,534,409
...,...,...,...,...,...,...,...,...
16214,2023-04-01,RKTU,RKPC,JNA,7,7,1281,1296
16215,2023-04-01,RKTU,RKPC,KAL,4,4,565,573
16216,2023-04-01,RKTU,RKPC,TWB,4,4,710,744
16217,2023-04-01,RCTP,RKPC,TWB,1,1,123,189


In [3]:
df["DEPART"].value_counts()
df["ARRIVE"].value_counts()
df["AIRLINE"].value_counts()

JNA    3838
JJA    3145
AAR    2291
TWB    2125
KAL    1855
ABL    1360
ASV     562
FGW     460
EOK     450
ESR     133
Name: AIRLINE, dtype: Int64

In [4]:
df2 = df.copy()

# create a dataframe with the ICAO to IATA mappings for airports
airport_mappings = pd.DataFrame({
    'ICAO': ['RKSS', 'RKSI', 'RKPC', 'RKPK', 'RKTU', 'RKJJ', 'RKTN', 'RKJB', 'RKNY', 'RKPU', 'RKPS'],
    'IATA': ['GMP', 'ICN', 'CJU', 'PUS', 'CJJ', 'KWJ', 'TAE', 'MWX', 'YNY', 'USN', 'HIN']
})

# create a dataframe with the ICAO to IATA mappings for airlines
airline_mappings = pd.DataFrame({
    "ICAO": ["KAL", "AAR", "JJA", "JNA", "ABL", "ESR", "TWI", "ASV", "TWB", "FGW", "EOK"],
    "IATA": ["KE", "OZ", "7C", "LJ", "BX", "ZE", "TW", "RS", "TW", "FG", "GJ"]

})

# merge the airport_mappings dataframe with the original dataframe to add the IATA codes for departure and arrival airports
df2 = pd.merge(df2, airport_mappings, how="left", left_on="DEPART", right_on="ICAO")
df2 = pd.merge(df2, airport_mappings, how="left", left_on="ARRIVE", right_on="ICAO", suffixes=("_DEPART", "_ARRIVE"))

# merge the airline_mappings dataframe with the original dataframe to add the IATA codes for airlines
df2 = pd.merge(df2, airline_mappings, how="left", left_on="AIRLINE", right_on="ICAO")

# drop the unnecessary columns
df2 = df2.drop(["ICAO_DEPART", "ICAO_ARRIVE", "ICAO"], axis=1)

df2

,DT,DEPART,ARRIVE,AIRLINE,AIR_DEPART,AIR_ARRIVE,PSG_DEPART,PSG_ARRIVE,IATA_DEPART,IATA_ARRIVE,IATA
0,2022-01-01,RKJJ,RKPC,AAR,3,3,427,330,KWJ,CJU,OZ
1,2022-01-01,RKJJ,RKPC,JJA,4,4,673,516,KWJ,CJU,7C
2,2022-01-01,RKJJ,RKPC,JNA,3,3,425,349,KWJ,CJU,LJ
3,2022-01-01,RKJJ,RKPC,KAL,2,2,202,180,KWJ,CJU,KE
4,2022-01-01,RKJJ,RKPC,TWB,3,3,534,409,KWJ,CJU,TW
...,...,...,...,...,...,...,...,...,...,...,...
16214,2023-04-01,RKTU,RKPC,JNA,7,7,1281,1296,CJJ,CJU,LJ
16215,2023-04-01,RKTU,RKPC,KAL,4,4,565,573,CJJ,CJU,KE
16216,2023-04-01,RKTU,RKPC,TWB,4,4,710,744,CJJ,CJU,TW
16217,2023-04-01,RCTP,RKPC,TWB,1,1,123,189,NaN,CJU,TW


In [5]:
# df2[df2["IATA_DEPART"].isnull()].value_counts("DEPART")
# df2[df2["IATA_DEPART"].isnull()].value_counts("DEPART")

In [6]:
df2.groupby(["DT", "ARRIVE"]).agg({"AIR_ARRIVE": "sum", "PSG_ARRIVE": "sum"})

,,AIR_ARRIVE,PSG_ARRIVE
DT,ARRIVE,,
2022-01-01,RKPC,229,29343
2022-01-02,RKPC,242,35494
2022-01-03,RKPC,227,37002
2022-01-04,RKPC,221,35380
2022-01-05,RKPC,219,36699
...,...,...,...
2023-03-28,RKPC,220,37386
2023-03-29,RKPC,228,38968
2023-03-30,RKPC,226,40319


In [7]:
df2.to_csv('./data/AIRPORTAL_CJU_2022.csv', index=False)